In [8]:
import os
import pdb
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, sampler

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import get_scheduler
from datasets import Dataset

from collections import Counter
import evaluate

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from sklearn.metrics import classification_report

In [29]:
model = RobertaForSequenceClassification.from_pretrained('roberta_no_priv_84/').to(device)

### Load data

In [ ]:
label_map = {'happy': 1, 'sad': 0}

# Download dataset from: https://github.com/mohummedalee/twitteraae-sentiment-data/
def load_twitter_aae(dir):
    sentences = []
    labels = []
    dialects = []
    # for dial in ['aae', 'sae']:
    for dial in ['aae']:
        # for lab in ['happy', 'sad']:
        for lab in ['happy']:
            # load dialect x sentiment combination
            fpath = os.path.join(dir, f'{dial}_{lab}')
            fh = open(fpath, 'r', encoding='utf-8')
            i = 0
            while True:                
                # print(i)
                try:
                    # pdb.set_trace()
                    line = fh.readline()
                    sentences.append(line.strip())
                    labels.append(label_map[lab])
                    dialects.append(dial.upper())
                    i += 1 
                except UnicodeDecodeError:
                    print(f'could not read {fpath} at line {i}')
                    continue
                except EOFError:
                    pdb.set_trace()
                    fh.close()
                    print(f'read {i} lines total from {fpath}')
                    break

    return sentences, labels, dialects

DATA_DIR = 'fairness-privacy-local/twitteraae-sentiment-data/data/raw/sentiment_race'
sentences, labels, dialects = load_twitter_aae(DATA_DIR)

# dataset = Dataset.from_dict({
#     'text': sentences,
#     'label': labels,
#     'dialect': dialects
# }).with_format("torch")

In [101]:
# how many pos/neg by dialect
aae_inds = [i for i in range(len(dataset)) if dataset['dialect'][i] == 'AAE']
sae_inds = [i for i in range(len(dataset)) if dataset['dialect'][i] == 'SAE']

print(len(aae_inds), len(sae_inds))

5750 6723


In [105]:
# compute label counts for both sae and aae
print('AAE:', Counter(
    [x.item() for x in dataset[aae_inds]['labels']]
))
print('SAE:', Counter(
    [x.item() for x in dataset[sae_inds]['labels']]
))

AAE: Counter({0: 5750})
SAE: Counter({0: 3625, 1: 3098})


In [12]:
# Tokenize data
MODEL_PATH = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

MAXLEN = 128

def tokenize(instance):
    tokenized = tokenizer(instance['text'], truncation=True, padding="max_length", max_length=MAXLEN)
    # return {**tokenized, "label": instance['label'], "dialect": instance['dialect']}
    return {**tokenized}
    
dataset = dataset.map(tokenize, num_proc=3)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=3):   0%|          | 0/12473 [00:00<?, ? examples/s]

/home/muh.ali/.conda/envs/fairness-privacy/lib/python3.11/site-packages/datasets/table.py:1381: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/muh.ali/.conda/envs/fairness-privacy/lib/python3.11/site-packages/datasets/table.py:1407: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [7]:
val_inds = np.loadtxt('fairness-privacy-local/val_inds.txt')
val_inds = [int(v) for v in val_inds]

In [16]:
Counter(dataset[val_inds]['dialect'])

Counter({'SAE': 681, 'AAE': 566})

### Evaluation

In [27]:
# train_sampler = sampler.SubsetRandomSampler(train_inds)
dataset = dataset.rename_column('label', 'labels')
val_sampler = sampler.SubsetRandomSampler(val_inds)
# test_sampler = sampler.SubsetRandomSampler(test_inds)
batch_size=64

# train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, sampler=val_sampler)

In [ ]:
# NOTE: should also know how cardiffnlp model does on SAE vs. AAE

In [55]:
# # overall 
evaluate_accuracy_inds(model, val_inds)

  0%|          | 0/39 [00:00<?, ?it/s]

{'accuracy': 0.8420208500400962}

In [44]:
# disaggregated by dialect
aae_inds_val = [i for i in val_inds if dataset['dialect'][i] == 'AAE']
sae_inds_val = [i for i in val_inds if dataset['dialect'][i] == 'SAE']

In [52]:
def evaluate_accuracy_inds(model, inds, batch_size=32):
    inds_sampler = sampler.SubsetRandomSampler(inds)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, sampler=inds_sampler)

    metric = evaluate.load('accuracy')
    model.eval()  # switch to eval mode
    for batch in tqdm(dataloader):
        batch_topass = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'labels': batch['labels'].to(device)
        }
        with torch.no_grad():
            outputs = model(**batch_topass)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        metric.add_batch(predictions=predictions, references=batch['labels'])
    
    return metric.compute()

In [77]:
def evaluate_for_sklearn(model, inds):
    # output: y_true, y_pred
    y_true, y_preds = [], []
    for i in tqdm(inds):
        y_true.append(dataset[i]['labels'].item())
        pt = {
            'input_ids': dataset[i:i+1]['input_ids'].to(device),
            'attention_mask': dataset[i:i+1]['attention_mask'].to(device),
            'labels': dataset[i:i+1]['labels'].to(device)
        }
        # pdb.set_trace()
        with torch.no_grad():
            outputs = model(**pt)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1)
        y_preds.append(prediction.item())

    return y_true, y_preds

In [82]:
aae_true, aae_preds = evaluate_for_sklearn(model, aae_inds_val)
print(classification_report(y_true, y_preds))

  0%|          | 0/566 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       1.00      0.92      0.96       566
           1       0.00      0.00      0.00         0

    accuracy                           0.92       566
   macro avg       0.50      0.46      0.48       566
weighted avg       1.00      0.92      0.96       566



/home/muh.ali/.conda/envs/fairness-privacy/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/muh.ali/.conda/envs/fairness-privacy/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/muh.ali/.conda/envs/fairness-privacy/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [84]:
dataset[aae_inds_val]['labels']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [53]:
evaluate_accuracy_inds(model, aae_inds_val)

  0%|          | 0/18 [00:00<?, ?it/s]

{'accuracy': 0.9187279151943463}

In [54]:
evaluate_accuracy_inds(model, sae_inds_val)

  0%|          | 0/22 [00:00<?, ?it/s]

{'accuracy': 0.7782672540381792}

### Debugging

In [85]:
train_inds = np.loadtxt('fairness-privacy-local/train_inds.txt')
train_inds = [int(v) for v in train_inds]

In [87]:
aae_inds_train = [i for i in train_inds if dataset['dialect'][i] == 'AAE']
sae_inds_train = [i for i in train_inds if dataset['dialect'][i] == 'SAE']

In [99]:
# [i.item() for i in dataset[aae_inds_train]['labels']]
dataset[aae_inds_train]['labels']

tensor([0, 0, 0,  ..., 0, 0, 0])

**NOTE**: 